# Decision Trees: Complete Lecture Series

## 📚 Amazon Applied Scientist Interview Preparation

### **Lecture Overview:**
This comprehensive lecture covers decision trees from mathematical foundations to production deployment, designed specifically for Applied Scientist interviews at top-tier companies.

---

## 🎯 **Learning Objectives**

By the end of this lecture, you will master:

### **1. Mathematical Foundations (25 minutes)**
- **Information Theory**: Entropy, Information Gain, Gini Impurity
- **Splitting Criteria**: Mathematical derivations and comparisons
- **Complexity Analysis**: Time/space complexity of tree operations

### **2. Algorithm Implementation (30 minutes)**
- **Tree Construction**: Recursive splitting algorithm
- **Pruning Techniques**: Pre-pruning vs post-pruning
- **Handling Mixed Data**: Categorical and numerical features

### **3. Advanced Concepts (25 minutes)**
- **Ensemble Methods**: Random Forest, Gradient Boosting integration
- **Feature Importance**: Calculation and interpretation
- **Bias-Variance Trade-off**: Overfitting prevention strategies

### **4. Production Considerations (20 minutes)**
- **Scalability**: Handling large datasets efficiently
- **Interpretability**: Business stakeholder communication
- **Real-world Applications**: Fraud detection, recommendation systems

---

## 📖 **Prerequisites**
- Basic probability and statistics
- Understanding of supervised learning
- Python programming proficiency

## 🎓 **Interview Relevance**
Decision trees are fundamental to many Amazon services:
- **Fraud Detection**: Rule-based interpretable models
- **Recommendation Systems**: Feature selection and ranking
- **A/B Testing**: Segmentation and analysis
- **Operational Decisions**: Automated business logic

---

# Lecture 1: Mathematical Foundations

## 🧮 **Information Theory Fundamentals**

### **Why Information Theory?**
Decision trees work by **reducing uncertainty** at each split. Information theory provides the mathematical framework to measure and optimize this uncertainty reduction.

### **Key Concepts:**

#### **1. Entropy (Claude Shannon, 1948)**
```
H(S) = -Σ p_i * log₂(p_i)
```

**Intuition**: Entropy measures the "surprise" or uncertainty in a dataset
- **H = 0**: Perfect purity (all samples same class)
- **H = 1**: Maximum uncertainty (binary classes, 50-50 split)
- **H = log₂(c)**: Maximum uncertainty for c classes

#### **2. Information Gain**
```
IG(S, A) = H(S) - Σ (|S_v|/|S|) * H(S_v)
```

**Intuition**: How much uncertainty does splitting on attribute A remove?

#### **3. Gini Impurity**
```
Gini(S) = 1 - Σ p_i²
```

**Intuition**: Probability of misclassifying a randomly chosen sample

---

### **🔍 Interview Question:**
*"Why do we use log₂ in entropy? What happens if we use natural log?"*

**Answer**: log₂ gives entropy in "bits" of information. Natural log gives "nats". The choice doesn't affect relative rankings of splits, only the absolute values.

In [ ]:
# Lecture 1: Mathematical Foundations Implementation

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from typing import List, Tuple, Optional, Dict, Any
import seaborn as sns
from sklearn.datasets import make_classification, load_iris
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)
plt.style.use('default')

print("📚 LECTURE 1: MATHEMATICAL FOUNDATIONS")
print("=" * 50)

class InformationTheory:
    """
    Core information theory calculations for decision trees.
    
    Essential for understanding:
    1. How trees choose optimal splits
    2. Why certain features are more informative
    3. Mathematical basis for pruning decisions
    
    Interview Focus:
    - Can you derive entropy formula?
    - When would Gini be preferred over entropy?
    - How does information gain handle continuous features?
    """
    
    @staticmethod
    def entropy(labels: np.ndarray) -> float:
        """
        Calculate Shannon entropy: H(S) = -Σ p_i * log₂(p_i)
        
        Args:
            labels: Array of class labels
            
        Returns:
            Entropy value in bits
            
        Time Complexity: O(n) where n is number of samples
        """
        if len(labels) == 0:
            return 0.0
        
        # Count class frequencies
        _, counts = np.unique(labels, return_counts=True)
        probabilities = counts / len(labels)
        
        # Calculate entropy (handle log(0) case)
        entropy = 0.0
        for p in probabilities:
            if p > 0:  # Avoid log(0)
                entropy -= p * np.log2(p)
        
        return entropy
    
    @staticmethod
    def gini_impurity(labels: np.ndarray) -> float:
        """
        Calculate Gini impurity: Gini(S) = 1 - Σ p_i²
        
        Intuition: Probability of misclassifying a randomly chosen sample
        
        Gini vs Entropy:
        - Gini: Faster to compute (no log)
        - Entropy: More theoretically principled
        - Both give similar results in practice
        """
        if len(labels) == 0:
            return 0.0
        
        _, counts = np.unique(labels, return_counts=True)
        probabilities = counts / len(labels)
        
        gini = 1.0 - np.sum(probabilities ** 2)
        return gini
    
    @staticmethod
    def information_gain(parent_labels: np.ndarray, 
                        child_labels_list: List[np.ndarray],
                        criterion: str = 'entropy') -> float:
        """
        Calculate information gain from a split.
        
        IG(S, A) = H(S) - Σ (|S_v|/|S|) * H(S_v)
        
        Args:
            parent_labels: Labels before split
            child_labels_list: List of label arrays after split
            criterion: 'entropy' or 'gini'
            
        Returns:
            Information gain (higher is better)
        """
        # Choose impurity function
        if criterion == 'entropy':
            impurity_fn = InformationTheory.entropy
        elif criterion == 'gini':
            impurity_fn = InformationTheory.gini_impurity
        else:
            raise ValueError(f"Unknown criterion: {criterion}")
        
        # Parent impurity
        parent_impurity = impurity_fn(parent_labels)
        
        # Weighted average of child impurities
        total_samples = len(parent_labels)
        weighted_child_impurity = 0.0
        
        for child_labels in child_labels_list:
            if len(child_labels) > 0:
                weight = len(child_labels) / total_samples
                child_impurity = impurity_fn(child_labels)
                weighted_child_impurity += weight * child_impurity
        
        information_gain = parent_impurity - weighted_child_impurity
        return information_gain
    
    @staticmethod
    def gain_ratio(parent_labels: np.ndarray, 
                   child_labels_list: List[np.ndarray]) -> float:
        """
        Calculate gain ratio to handle bias toward high-cardinality features.
        
        GainRatio(S, A) = InformationGain(S, A) / SplitInformation(S, A)
        
        Used in C4.5 algorithm to prevent overfitting to features 
        with many unique values.
        """
        info_gain = InformationTheory.information_gain(parent_labels, child_labels_list)
        
        # Calculate split information (entropy of split sizes)
        total_samples = len(parent_labels)
        split_sizes = [len(child) for child in child_labels_list if len(child) > 0]
        
        if not split_sizes:
            return 0.0
        
        split_info = 0.0
        for size in split_sizes:
            if size > 0:
                p = size / total_samples
                split_info -= p * np.log2(p)
        
        # Avoid division by zero
        if split_info == 0:
            return 0.0
        
        return info_gain / split_info


# Demonstrate information theory concepts
print("\n🧮 INFORMATION THEORY DEMONSTRATION:")

# Create sample datasets with different purities
datasets = {
    'Perfect Purity': np.array([1, 1, 1, 1, 1]),
    'Maximum Uncertainty (Binary)': np.array([0, 1, 0, 1, 0, 1]),
    'Maximum Uncertainty (3-class)': np.array([0, 1, 2, 0, 1, 2, 0, 1, 2]),
    'Slight Bias': np.array([0, 0, 0, 1, 1]),
    'Heavy Bias': np.array([0, 0, 0, 0, 1])
}

info_theory = InformationTheory()

print("\nDataset Impurity Analysis:")
print(f"{'Dataset':<30} {'Entropy':<10} {'Gini':<10} {'Classes':<15}")
print("-" * 70)

for name, labels in datasets.items():
    entropy_val = info_theory.entropy(labels)
    gini_val = info_theory.gini_impurity(labels)
    unique_classes = len(np.unique(labels))
    
    print(f"{name:<30} {entropy_val:<10.3f} {gini_val:<10.3f} {unique_classes:<15}")

print(f"\n💡 Key Insights:")
print(f"• Perfect purity: Entropy = Gini = 0")
print(f"• Maximum binary uncertainty: Entropy ≈ 1.0, Gini = 0.5")
print(f"• Maximum 3-class uncertainty: Entropy ≈ 1.58, Gini ≈ 0.67")
print(f"• Entropy grows as log₂(classes), Gini approaches (c-1)/c")

# Demonstrate information gain calculation
print("\n📊 INFORMATION GAIN EXAMPLE:")

# Sample split scenario
parent = np.array([0, 0, 1, 1, 1, 1, 0, 0])  # 4 class-0, 4 class-1
child1 = np.array([0, 0, 0])  # Left child: 3 class-0
child2 = np.array([1, 1, 1, 1, 0])  # Right child: 4 class-1, 1 class-0

ig_entropy = info_theory.information_gain(parent, [child1, child2], 'entropy')
ig_gini = info_theory.information_gain(parent, [child1, child2], 'gini')
gr = info_theory.gain_ratio(parent, [child1, child2])

print(f"Parent entropy: {info_theory.entropy(parent):.3f}")
print(f"Child 1 entropy: {info_theory.entropy(child1):.3f}")
print(f"Child 2 entropy: {info_theory.entropy(child2):.3f}")
print(f"Information Gain (Entropy): {ig_entropy:.3f}")
print(f"Information Gain (Gini): {ig_gini:.3f}")
print(f"Gain Ratio: {gr:.3f}")

print(f"\n✅ Mathematical foundations established!")

---

# Lecture 2: Decision Tree Algorithm Implementation

## 🌳 **Tree Construction Algorithm**

### **Core Algorithm: ID3/C4.5/CART**

```python
def build_tree(data, target, features):
    # Base cases
    if all_same_class(target):
        return LeafNode(target[0])
    
    if no_features_left(features):
        return LeafNode(majority_class(target))
    
    # Find best split
    best_feature, best_threshold = find_best_split(data, target, features)
    
    # Create splits
    left_data, right_data = split_data(data, best_feature, best_threshold)
    
    # Recursive construction
    left_subtree = build_tree(left_data, ...)
    right_subtree = build_tree(right_data, ...)
    
    return InternalNode(best_feature, best_threshold, left_subtree, right_subtree)
```

### **Key Design Decisions:**

#### **1. Stopping Criteria**
- **Pure nodes**: All samples same class
- **Minimum samples**: Prevent overfitting
- **Maximum depth**: Computational/memory limits
- **Minimum improvement**: Information gain threshold

#### **2. Handling Continuous Features**
- **Binary splits**: feature ≤ threshold
- **Threshold selection**: Sort values, try midpoints
- **Computational cost**: O(n log n) per feature

#### **3. Missing Values**
- **Surrogate splits**: Use correlated features
- **Probabilistic assignment**: Distribute samples proportionally
- **Separate category**: Treat missing as distinct value

---

### **🔍 Interview Question:**
*"How do you handle a feature with 1000 unique values? What's the computational complexity?"*

**Answer**: For continuous features, sort once (O(n log n)) then evaluate n-1 possible thresholds. For categorical features with high cardinality, consider grouping or using gain ratio to prevent bias.

In [ ]:
# Lecture 2: Decision Tree Implementation

from dataclasses import dataclass
from abc import ABC, abstractmethod

print("📚 LECTURE 2: DECISION TREE ALGORITHM")
print("=" * 50)

@dataclass
class TreeNode(ABC):
    """Abstract base class for tree nodes."""
    pass

@dataclass 
class LeafNode(TreeNode):
    """Leaf node containing prediction."""
    prediction: Any
    samples: int = 0
    confidence: float = 1.0
    
@dataclass
class InternalNode(TreeNode):
    """Internal node containing split criteria."""
    feature_idx: int
    threshold: float
    left: TreeNode
    right: TreeNode
    samples: int = 0
    impurity: float = 0.0


class DecisionTreeClassifier:
    """
    Complete decision tree implementation with all production features.
    
    Features:
    1. Information gain and Gini splitting criteria
    2. Pruning (pre and post)
    3. Feature importance calculation
    4. Handle mixed data types
    5. Missing value handling
    
    Interview Focus:
    - Time complexity analysis
    - Memory usage optimization
    - Overfitting prevention strategies
    - Real-world edge cases
    """
    
    def __init__(self, 
                 criterion: str = 'gini',
                 max_depth: Optional[int] = None,
                 min_samples_split: int = 2,
                 min_samples_leaf: int = 1,
                 min_impurity_decrease: float = 0.0,
                 random_state: Optional[int] = None):
        """
        Initialize decision tree classifier.
        
        Parameters follow sklearn convention for familiarity.
        """
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.min_impurity_decrease = min_impurity_decrease
        self.random_state = random_state
        
        # Set after fitting
        self.tree_ = None
        self.n_features_ = None
        self.classes_ = None
        self.feature_importances_ = None
        
        if random_state is not None:
            np.random.seed(random_state)
    
    def fit(self, X: np.ndarray, y: np.ndarray) -> 'DecisionTreeClassifier':
        """
        Build decision tree from training data.
        
        Time Complexity: O(n * m * log(n)) where n=samples, m=features
        Space Complexity: O(n) in worst case (skewed tree)
        """
        # Validate input
        X = np.array(X)
        y = np.array(y)
        
        if X.shape[0] != y.shape[0]:
            raise ValueError("X and y must have same number of samples")
        
        # Store dataset properties
        self.n_features_ = X.shape[1]
        self.classes_ = np.unique(y)
        
        # Initialize feature importances
        self.feature_importances_ = np.zeros(self.n_features_)
        
        # Build tree recursively
        self.tree_ = self._build_tree(X, y, depth=0)
        
        # Normalize feature importances
        if np.sum(self.feature_importances_) > 0:
            self.feature_importances_ /= np.sum(self.feature_importances_)
        
        return self
    
    def _build_tree(self, X: np.ndarray, y: np.ndarray, depth: int) -> TreeNode:
        """
        Recursive tree building algorithm.
        
        Core of the decision tree algorithm - this is what interviewers
        want to see you implement from scratch.
        """
        n_samples, n_features = X.shape
        
        # Calculate current impurity
        current_impurity = self._calculate_impurity(y)
        
        # Check stopping criteria
        if (self._should_stop_splitting(X, y, depth, current_impurity)):
            return self._create_leaf(y)
        
        # Find best split
        best_split = self._find_best_split(X, y)
        
        if best_split is None:
            return self._create_leaf(y)
        
        feature_idx, threshold, left_mask, impurity_decrease = best_split
        
        # Check minimum impurity decrease
        if impurity_decrease < self.min_impurity_decrease:
            return self._create_leaf(y)
        
        # Update feature importance
        self.feature_importances_[feature_idx] += (
            (n_samples / len(y)) * impurity_decrease
        )
        
        # Create splits
        X_left, y_left = X[left_mask], y[left_mask]
        X_right, y_right = X[~left_mask], y[~left_mask]
        
        # Recursive calls
        left_subtree = self._build_tree(X_left, y_left, depth + 1)
        right_subtree = self._build_tree(X_right, y_right, depth + 1)
        
        return InternalNode(
            feature_idx=feature_idx,
            threshold=threshold,
            left=left_subtree,
            right=right_subtree,
            samples=n_samples,
            impurity=current_impurity
        )
    
    def _should_stop_splitting(self, X: np.ndarray, y: np.ndarray, 
                              depth: int, impurity: float) -> bool:
        """
        Check various stopping criteria.
        
        Critical for preventing overfitting in production.
        """
        n_samples = len(y)
        
        # Pure node (all same class)
        if impurity == 0:
            return True
        
        # Maximum depth reached
        if self.max_depth is not None and depth >= self.max_depth:
            return True
        
        # Minimum samples for split
        if n_samples < self.min_samples_split:
            return True
        
        # Single class remaining
        if len(np.unique(y)) == 1:
            return True
        
        return False
    
    def _find_best_split(self, X: np.ndarray, y: np.ndarray) -> Optional[Tuple]:
        """
        Find the best feature and threshold to split on.
        
        This is the computational bottleneck of tree construction.
        
        Returns:
            Tuple of (feature_idx, threshold, left_mask, impurity_decrease)
            or None if no valid split found
        """
        n_samples, n_features = X.shape
        
        if n_samples <= 1:
            return None
        
        # Current impurity
        parent_impurity = self._calculate_impurity(y)
        
        best_gain = -1
        best_split = None
        
        # Try each feature
        for feature_idx in range(n_features):
            feature_values = X[:, feature_idx]
            
            # Get potential thresholds (sorted unique values)
            unique_values = np.unique(feature_values)
            
            if len(unique_values) <= 1:
                continue
            
            # Try thresholds between consecutive unique values
            for i in range(len(unique_values) - 1):
                threshold = (unique_values[i] + unique_values[i + 1]) / 2
                
                # Create split
                left_mask = feature_values <= threshold
                
                # Check minimum samples in leaves
                if (np.sum(left_mask) < self.min_samples_leaf or 
                    np.sum(~left_mask) < self.min_samples_leaf):
                    continue
                
                # Calculate information gain
                gain = self._calculate_information_gain(
                    y, y[left_mask], y[~left_mask], parent_impurity
                )
                
                if gain > best_gain:
                    best_gain = gain
                    best_split = (feature_idx, threshold, left_mask, gain)
        
        return best_split
    
    def _calculate_impurity(self, y: np.ndarray) -> float:
        """
        Calculate impurity using specified criterion.
        """
        if self.criterion == 'gini':
            return InformationTheory.gini_impurity(y)
        elif self.criterion == 'entropy':
            return InformationTheory.entropy(y)
        else:
            raise ValueError(f"Unknown criterion: {self.criterion}")
    
    def _calculate_information_gain(self, parent: np.ndarray, 
                                   left: np.ndarray, right: np.ndarray,
                                   parent_impurity: float) -> float:
        """
        Calculate information gain from a split.
        """
        n_parent = len(parent)
        n_left = len(left)
        n_right = len(right)
        
        if n_left == 0 or n_right == 0:
            return 0
        
        # Weighted impurity of children
        left_impurity = self._calculate_impurity(left)
        right_impurity = self._calculate_impurity(right)
        
        child_impurity = (
            (n_left / n_parent) * left_impurity +
            (n_right / n_parent) * right_impurity
        )
        
        return parent_impurity - child_impurity
    
    def _create_leaf(self, y: np.ndarray) -> LeafNode:
        """
        Create leaf node with majority class prediction.
        """
        classes, counts = np.unique(y, return_counts=True)
        majority_class = classes[np.argmax(counts)]
        confidence = np.max(counts) / len(y)
        
        return LeafNode(
            prediction=majority_class,
            samples=len(y),
            confidence=confidence
        )
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Make predictions for input samples.
        
        Time Complexity: O(log n) per sample for balanced tree
        Space Complexity: O(1) per prediction
        """
        if self.tree_ is None:
            raise ValueError("Tree not fitted. Call fit() first.")
        
        X = np.array(X)
        predictions = []
        
        for sample in X:
            prediction = self._predict_sample(sample, self.tree_)
            predictions.append(prediction)
        
        return np.array(predictions)
    
    def _predict_sample(self, sample: np.ndarray, node: TreeNode) -> Any:
        """
        Traverse tree to make prediction for single sample.
        """
        if isinstance(node, LeafNode):
            return node.prediction
        
        # Internal node - follow appropriate branch
        if sample[node.feature_idx] <= node.threshold:
            return self._predict_sample(sample, node.left)
        else:
            return self._predict_sample(sample, node.right)
    
    def get_depth(self) -> int:
        """
        Calculate depth of the tree.
        """
        if self.tree_ is None:
            return 0
        return self._calculate_depth(self.tree_)
    
    def _calculate_depth(self, node: TreeNode) -> int:
        """
        Recursively calculate tree depth.
        """
        if isinstance(node, LeafNode):
            return 1
        
        left_depth = self._calculate_depth(node.left)
        right_depth = self._calculate_depth(node.right)
        
        return 1 + max(left_depth, right_depth)


# Demonstration of decision tree implementation
print("\n🌳 DECISION TREE IMPLEMENTATION DEMO:")

# Create sample dataset
X, y = make_classification(
    n_samples=200,
    n_features=4,
    n_redundant=0,
    n_informative=4,
    n_clusters_per_class=1,
    random_state=42
)

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print(f"Dataset: {X_train.shape[0]} train, {X_test.shape[0]} test samples")
print(f"Features: {X_train.shape[1]}, Classes: {len(np.unique(y_train))}")

# Train decision tree
dt = DecisionTreeClassifier(
    criterion='gini',
    max_depth=5,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42
)

dt.fit(X_train, y_train)

# Make predictions
train_pred = dt.predict(X_train)
test_pred = dt.predict(X_test)

# Calculate accuracy
train_acc = np.mean(train_pred == y_train)
test_acc = np.mean(test_pred == y_test)

print(f"\n📊 RESULTS:")
print(f"Tree depth: {dt.get_depth()}")
print(f"Training accuracy: {train_acc:.3f}")
print(f"Testing accuracy: {test_acc:.3f}")
print(f"Overfitting gap: {train_acc - test_acc:.3f}")

# Feature importance
print(f"\n🔍 FEATURE IMPORTANCE:")
for i, importance in enumerate(dt.feature_importances_):
    print(f"Feature {i}: {importance:.3f}")

print(f"\n✅ Decision tree algorithm implemented successfully!")

---

# Lecture 3: Advanced Concepts

## 🌲 **Ensemble Methods Integration**

### **Why Single Trees Fail**
- **High Variance**: Small data changes → completely different trees
- **Overfitting**: Complex trees memorize training noise
- **Limited Expressiveness**: Single tree can't capture complex patterns

### **Random Forest Foundation**
```
Random Forest = Bootstrap Aggregating + Random Feature Selection
```

**Key Ideas:**
1. **Bagging**: Train multiple trees on bootstrap samples
2. **Feature Randomness**: Each split considers √m random features
3. **Voting**: Average predictions across trees

**Mathematical Foundation:**
```
Variance(avg) = Variance(individual) / n    (if uncorrelated)
```

### **Gradient Boosting Connection**
- **Sequential Learning**: Each tree corrects previous errors
- **Weak Learners**: Shallow trees (stumps) work best
- **Additive Model**: F(x) = Σ α_i * h_i(x)

---

## 🎯 **Feature Importance Deep Dive**

### **Calculation Methods:**

#### **1. Impurity-Based Importance**
```
Importance(feature) = Σ (n_samples/total) * impurity_decrease
```

#### **2. Permutation Importance**
```
Importance = Score_original - Score_permuted
```

#### **3. Drop-Column Importance**
```
Importance = Score_with_feature - Score_without_feature
```

### **⚠️ Bias Issues:**
- **High-cardinality bias**: Features with more values get higher importance
- **Correlated features**: Importance shared unpredictably
- **Tree structure dependency**: Deep vs shallow affects calculations

---

## 📊 **Bias-Variance Trade-off**

### **Decision Tree Bias-Variance Profile:**
- **High Variance**: Different training sets → very different trees
- **Low Bias**: Can represent complex decision boundaries
- **Overfitting Tendency**: Without constraints, memorizes training data

### **Regularization Strategies:**
1. **Pre-pruning**: Stop early (depth, samples, impurity)
2. **Post-pruning**: Build full tree, then remove branches
3. **Ensembles**: Reduce variance through averaging

In [ ]:
# Lecture 3: Advanced Concepts Implementation

print("📚 LECTURE 3: ADVANCED CONCEPTS")
print("=" * 50)

class PruningMethods:
    """
    Advanced pruning techniques for decision trees.
    
    Critical for production deployment:
    1. Prevents overfitting
    2. Reduces model size
    3. Improves interpretability
    4. Faster inference
    
    Interview Focus:
    - Cost complexity pruning algorithm
    - When to use pre vs post pruning
    - How pruning affects bias-variance
    """
    
    @staticmethod
    def cost_complexity_pruning(tree: DecisionTreeClassifier, X_val: np.ndarray, 
                               y_val: np.ndarray) -> DecisionTreeClassifier:
        """
        Post-pruning using cost complexity (minimal cost-complexity pruning).
        
        Algorithm:
        1. Calculate cost-complexity for each subtree
        2. Find weakest link (smallest alpha increase)
        3. Prune weakest link
        4. Repeat until only root remains
        5. Select best tree using validation set
        
        Used in CART algorithm.
        """
        # Simplified implementation - in practice this is quite complex
        print("🌿 Cost Complexity Pruning (Conceptual)")
        print("Real implementation requires tracking all pruning sequences")
        
        # For demonstration, we'll show the concept
        best_score = 0
        best_tree = tree
        
        # Would iterate through all possible prunings
        # and select the one with best validation performance
        
        return best_tree

class FeatureImportanceAnalyzer:
    """
    Comprehensive feature importance analysis.
    
    Multiple methods to understand feature contributions:
    1. Impurity-based (built into tree)
    2. Permutation importance (model-agnostic)
    3. Drop-column importance (most reliable)
    4. SHAP values (modern approach)
    """
    
    @staticmethod
    def permutation_importance(model, X: np.ndarray, y: np.ndarray, 
                             n_repeats: int = 10, random_state: int = 42) -> Dict:
        """
        Calculate permutation importance for each feature.
        
        Algorithm:
        1. Calculate baseline score
        2. For each feature:
           a. Shuffle feature values
           b. Calculate score decrease
           c. Repeat n_repeats times
        3. Return mean and std of importance
        
        Advantages:
        - Model agnostic
        - Captures feature interactions
        - Less biased than impurity-based
        """
        np.random.seed(random_state)
        
        # Baseline score
        baseline_score = np.mean(model.predict(X) == y)
        
        importances = {}\n",
        
        for feature_idx in range(X.shape[1]):
            feature_scores = []
            
            for _ in range(n_repeats):
                # Copy data and shuffle one feature
                X_permuted = X.copy()
                np.random.shuffle(X_permuted[:, feature_idx])
                
                # Calculate score with permuted feature
                permuted_score = np.mean(model.predict(X_permuted) == y)
                feature_scores.append(baseline_score - permuted_score)
            
            importances[f'feature_{feature_idx}'] = {
                'mean': np.mean(feature_scores),
                'std': np.std(feature_scores)
            }
        
        return importances
    
    @staticmethod
    def drop_column_importance(model_class, X: np.ndarray, y: np.ndarray, 
                             **model_kwargs) -> Dict:
        """
        Calculate drop-column importance.
        
        Most reliable but computationally expensive:
        - Trains n+1 models (baseline + one per feature)
        - Measures performance drop when feature removed
        - Accounts for feature interactions properly
        """
        # Baseline model with all features
        baseline_model = model_class(**model_kwargs)
        baseline_model.fit(X, y)
        baseline_score = np.mean(baseline_model.predict(X) == y)
        
        importances = {}
        
        for feature_idx in range(X.shape[1]):
            # Create dataset without this feature
            X_reduced = np.delete(X, feature_idx, axis=1)
            
            # Train model without feature
            reduced_model = model_class(**model_kwargs)
            reduced_model.fit(X_reduced, y)
            reduced_score = np.mean(reduced_model.predict(X_reduced) == y)
            
            # Importance is performance drop
            importance = baseline_score - reduced_score
            importances[f'feature_{feature_idx}'] = importance
        
        return importances

class BiasVarianceAnalysis:
    """
    Empirical bias-variance decomposition for decision trees.
    
    Critical for understanding model behavior:
    - Bias: How far predictions are from true values
    - Variance: How much predictions vary across training sets
    - Noise: Irreducible error in the problem
    
    Total Error = Bias² + Variance + Noise
    """
    
    @staticmethod
    def decompose_error(model_class, X: np.ndarray, y: np.ndarray, 
                       n_trials: int = 100, test_size: float = 0.3,
                       **model_kwargs) -> Dict:
        """
        Empirical bias-variance decomposition.
        
        Algorithm:
        1. Generate many bootstrap samples
        2. Train model on each sample
        3. Predict on fixed test set
        4. Calculate bias and variance empirically
        """
        np.random.seed(42)
        
        # Split data once to get fixed test set
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=42
        )
        
        predictions = []\n",
        
        # Train multiple models on bootstrap samples
        for trial in range(n_trials):
            # Bootstrap sample
            n_samples = len(X_train)
            bootstrap_indices = np.random.choice(n_samples, n_samples, replace=True)
            X_boot = X_train[bootstrap_indices]
            y_boot = y_train[bootstrap_indices]
            
            # Train model
            model = model_class(**model_kwargs)
            model.fit(X_boot, y_boot)
            
            # Predict on test set
            pred = model.predict(X_test)
            predictions.append(pred)
        
        predictions = np.array(predictions)
        
        # Calculate bias and variance for each test point
        mean_prediction = np.mean(predictions, axis=0)
        
        # For classification, we'll use probability of correct prediction
        correct_predictions = (predictions == y_test[np.newaxis, :]).astype(float)
        
        bias_squared = np.mean((mean_prediction - y_test) ** 2)
        variance = np.mean(np.var(correct_predictions, axis=0))
        
        # Test error
        test_predictions = []
        for trial in range(10):  # Smaller sample for test error
            model = model_class(**model_kwargs)
            model.fit(X_train, y_train)
            test_predictions.append(model.predict(X_test))
        
        test_error = 1 - np.mean([np.mean(pred == y_test) for pred in test_predictions])
        
        return {
            'bias_squared': float(bias_squared),
            'variance': float(variance),
            'test_error': float(test_error),
            'bias_variance_sum': float(bias_squared + variance)
        }

# Demonstrate advanced concepts
print("\n🌿 PRUNING DEMONSTRATION:")

# Create a dataset prone to overfitting
X_complex, y_complex = make_classification(
    n_samples=300,
    n_features=20,
    n_informative=5,
    n_redundant=10,
    n_clusters_per_class=1,
    flip_y=0.1,  # Add noise
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X_complex, y_complex, test_size=0.3, random_state=42
)

# Train unpruned tree (likely to overfit)
unpruned_tree = DecisionTreeClassifier(
    criterion='gini',
    max_depth=None,  # No depth limit
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)
unpruned_tree.fit(X_train, y_train)

# Train pruned tree
pruned_tree = DecisionTreeClassifier(
    criterion='gini',
    max_depth=5,
    min_samples_split=20,
    min_samples_leaf=10,
    random_state=42
)
pruned_tree.fit(X_train, y_train)

# Compare performance
unpruned_train_acc = np.mean(unpruned_tree.predict(X_train) == y_train)
unpruned_test_acc = np.mean(unpruned_tree.predict(X_test) == y_test)

pruned_train_acc = np.mean(pruned_tree.predict(X_train) == y_train)
pruned_test_acc = np.mean(pruned_tree.predict(X_test) == y_test)

print(f"Unpruned Tree:")
print(f"  Depth: {unpruned_tree.get_depth()}")
print(f"  Train Accuracy: {unpruned_train_acc:.3f}")
print(f"  Test Accuracy: {unpruned_test_acc:.3f}")
print(f"  Overfitting: {unpruned_train_acc - unpruned_test_acc:.3f}")

print(f"\nPruned Tree:")
print(f"  Depth: {pruned_tree.get_depth()}")
print(f"  Train Accuracy: {pruned_train_acc:.3f}")
print(f"  Test Accuracy: {pruned_test_acc:.3f}")
print(f"  Overfitting: {pruned_train_acc - pruned_test_acc:.3f}")

print(f"\n🔍 FEATURE IMPORTANCE ANALYSIS:")

# Impurity-based importance (from tree)
print("Impurity-based importance (top 5 features):")
importance_indices = np.argsort(unpruned_tree.feature_importances_)[::-1][:5]
for i, idx in enumerate(importance_indices):
    print(f"  Feature {idx}: {unpruned_tree.feature_importances_[idx]:.3f}")

# Permutation importance
analyzer = FeatureImportanceAnalyzer()
perm_importance = analyzer.permutation_importance(unpruned_tree, X_test, y_test)

print(f"\nPermutation importance (top 5 features):")
sorted_perm = sorted(perm_importance.items(), 
                    key=lambda x: x[1]['mean'], reverse=True)[:5]
for feature, scores in sorted_perm:
    print(f"  {feature}: {scores['mean']:.3f} (±{scores['std']:.3f})")

print(f"\n📊 BIAS-VARIANCE ANALYSIS:")

# Analyze bias-variance for different tree complexities
bias_var_analyzer = BiasVarianceAnalysis()

# Simple tree (high bias, low variance)
simple_results = bias_var_analyzer.decompose_error(
    DecisionTreeClassifier,
    X_complex, y_complex,
    n_trials=50,
    max_depth=3,
    min_samples_split=50,
    random_state=42
)

# Complex tree (low bias, high variance)
complex_results = bias_var_analyzer.decompose_error(
    DecisionTreeClassifier,
    X_complex, y_complex,
    n_trials=50,
    max_depth=None,
    min_samples_split=2,
    random_state=42
)

print(f"Simple Tree (max_depth=3):")
print(f"  Bias²: {simple_results['bias_squared']:.3f}")
print(f"  Variance: {simple_results['variance']:.3f}")
print(f"  Test Error: {simple_results['test_error']:.3f}")

print(f"\nComplex Tree (no limits):")
print(f"  Bias²: {complex_results['bias_squared']:.3f}")
print(f"  Variance: {complex_results['variance']:.3f}")
print(f"  Test Error: {complex_results['test_error']:.3f}")

print(f"\n💡 Key Insights:")
print(f"• Simple trees: Higher bias, lower variance")
print(f"• Complex trees: Lower bias, higher variance")
print(f"• Optimal complexity balances both")
print(f"• Ensembles can reduce variance while maintaining low bias")

print(f"\n✅ Advanced concepts mastered!")

---

# Lecture 4: Production Considerations

## 🚀 **Scalability & Performance**

### **Computational Complexity Analysis**

#### **Training Complexity:**
- **Best Case**: O(n * m * log(n)) - balanced splits
- **Worst Case**: O(n² * m) - skewed tree (linear chain)
- **Average Case**: O(n * m * log(n)) for most real datasets

Where: n = samples, m = features

#### **Memory Complexity:**
- **Tree Storage**: O(nodes) ≈ O(n) in worst case
- **Training Memory**: O(n * m) for feature sorting
- **Prediction**: O(1) per sample

### **Scaling Strategies:**

#### **1. Large Dataset Handling**
```python
# Streaming/Mini-batch approaches
# - Hoeffding Trees (VFDT)
# - Incremental learning
# - Approximate splits
```

#### **2. High-Dimensional Data**
```python
# Feature selection techniques
# - Random subspace sampling
# - Statistical tests (chi-square, mutual info)
# - Recursive feature elimination
```

#### **3. Distributed Computing**
```python
# Parallel tree construction
# - Feature-parallel: Different workers handle different features
# - Data-parallel: Split data across workers
# - Model-parallel: Different subtrees on different machines
```

---

## 📈 **Interpretability & Explainability**

### **Why Decision Trees Excel at Interpretability:**

1. **White-box Model**: Complete decision path visible
2. **Natural Language**: Rules easily translated to business logic
3. **Feature Interactions**: Explicit in tree structure
4. **Confidence Measures**: Sample counts at leaves

### **Business Communication Strategies:**

#### **Rule Extraction:**
```
IF feature_1 <= 3.5 AND feature_2 > 1.2 THEN class = A (confidence: 85%)
```

#### **Visual Representations:**
- Tree diagrams with business metrics
- Feature importance rankings
- Decision boundary plots

#### **Stakeholder-Friendly Metrics:**
- **Precision/Recall** instead of accuracy
- **Business impact** per decision rule
- **Confidence intervals** for predictions

---

## 🎯 **Real-World Applications**

### **Amazon Use Cases:**

#### **1. Fraud Detection**
```
Decision Rules:
├── transaction_amount > $1000
│   ├── account_age < 30 days → HIGH RISK (review manually)
│   └── account_age >= 30 days
│       ├── unusual_location = True → MEDIUM RISK
│       └── unusual_location = False → LOW RISK
└── transaction_amount <= $1000 → AUTO-APPROVE
```

**Advantages:**
- Explainable decisions for compliance
- Fast inference for real-time scoring
- Easy to update rules based on new fraud patterns

#### **2. Recommendation Systems**
```
User Segmentation Tree:
├── purchase_history > 50 items
│   ├── avg_rating >= 4.0 → "Premium Customer" (collaborative filtering)
│   └── avg_rating < 4.0 → "Active Bargain Hunter" (price-based recs)
└── purchase_history <= 50 items
    ├── age < 25 → "Young Explorer" (trending items)
    └── age >= 25 → "Cautious Buyer" (high-rated items)
```

#### **3. A/B Testing Analysis**
```
Treatment Effect Tree:
├── user_segment = "power_user"
│   ├── device = "mobile" → treatment_effect = +15%
│   └── device = "desktop" → treatment_effect = +5%
└── user_segment = "casual"
    └── device = "mobile" → treatment_effect = -2%
```

---

## ⚠️ **Production Pitfalls & Solutions**

### **Common Issues:**

#### **1. Data Drift**
**Problem**: Tree trained on old data makes poor decisions on new data
**Solution**: 
- Monitor feature distributions
- Retrain periodically
- Online learning algorithms

#### **2. Categorical Feature Explosion**
**Problem**: High-cardinality categories create overfitting
**Solutions**:
- Feature hashing
- Frequency-based encoding
- Target encoding with regularization

#### **3. Missing Value Handling**
**Problem**: Production data has missing values not seen in training
**Solutions**:
- Surrogate splits
- Separate "missing" category
- Multiple imputation strategies

#### **4. Model Staleness**
**Problem**: Business rules change faster than model updates
**Solutions**:
- Automated retraining pipelines
- A/B testing framework
- Rule override mechanisms

In [ ]:
# Lecture 4: Production Implementation

from sklearn.tree import export_text
from sklearn.preprocessing import LabelEncoder
import time
from memory_profiler import profile
import sys

print("📚 LECTURE 4: PRODUCTION CONSIDERATIONS")
print("=" * 50)

class ProductionDecisionTree:
    """
    Production-ready decision tree with all enterprise features.
    
    Key Production Features:
    1. Performance monitoring
    2. Missing value handling
    3. Categorical encoding
    4. Rule extraction
    5. Model versioning
    6. Drift detection
    
    Interview Focus:
    - How to handle production edge cases
    - Scaling strategies for large datasets
    - Business stakeholder communication
    - MLOps integration patterns
    """
    
    def __init__(self, **kwargs):
        self.model = DecisionTreeClassifier(**kwargs)
        self.label_encoders = {}
        self.feature_names = None
        self.training_stats = {}
        self.is_fitted = False
        
    def fit(self, X, y, feature_names=None):
        """
        Fit with production monitoring and preprocessing.
        """
        start_time = time.time()
        
        # Store feature names for interpretability
        if feature_names is not None:
            self.feature_names = feature_names
        else:
            self.feature_names = [f"feature_{i}" for i in range(X.shape[1])]
        
        # Handle categorical features (if any)
        X_processed = self._preprocess_features(X, fit=True)
        
        # Fit model
        self.model.fit(X_processed, y)
        self.is_fitted = True
        
        # Store training statistics
        training_time = time.time() - start_time
        self.training_stats = {
            'training_time': training_time,
            'n_samples': X.shape[0],
            'n_features': X.shape[1],
            'tree_depth': self.model.get_depth(),
            'n_classes': len(np.unique(y))
        }
        
        return self
    
    def _preprocess_features(self, X, fit=False):
        """
        Handle categorical features and missing values.
        """
        X_processed = X.copy()
        
        # In a real implementation, you'd detect categorical columns
        # For demonstration, assume all features are numeric
        
        # Handle missing values (replace with median)
        if fit:
            self.feature_medians = np.nanmedian(X_processed, axis=0)
        
        # Fill missing values
        for col in range(X_processed.shape[1]):
            mask = np.isnan(X_processed[:, col])
            if np.any(mask):
                X_processed[mask, col] = self.feature_medians[col]
        
        return X_processed
    
    def predict(self, X):
        """
        Predict with preprocessing and monitoring.
        """
        if not self.is_fitted:
            raise ValueError("Model not fitted. Call fit() first.")
        
        X_processed = self._preprocess_features(X, fit=False)
        return self.model.predict(X_processed)
    
    def extract_rules(self, max_depth=None):
        """
        Extract human-readable decision rules.
        
        Critical for business stakeholder communication.
        """
        if not self.is_fitted:
            raise ValueError("Model not fitted. Call fit() first.")
        
        # Use sklearn's text export (simplified version)
        rules = export_text(
            self.model,
            feature_names=self.feature_names,
            max_depth=max_depth
        )
        
        return rules
    
    def get_feature_importance_report(self):
        """
        Generate comprehensive feature importance report.
        """
        if not self.is_fitted:
            raise ValueError("Model not fitted. Call fit() first.")
        
        importances = self.model.feature_importances_
        
        # Create report
        report = []
        for i, importance in enumerate(importances):
            if importance > 0.01:  # Filter out very low importance
                report.append({
                    'feature': self.feature_names[i],
                    'importance': importance,
                    'rank': np.sum(importances > importance) + 1
                })
        
        # Sort by importance
        report.sort(key=lambda x: x['importance'], reverse=True)
        
        return report
    
    def performance_profile(self):
        """
        Return performance characteristics for monitoring.
        """
        return {
            'model_size_bytes': sys.getsizeof(self.model),
            'tree_depth': self.model.get_depth(),
            'n_leaves': self._count_leaves(),
            'training_stats': self.training_stats
        }
    
    def _count_leaves(self):
        """Count number of leaf nodes."""
        def count_leaves_recursive(node):
            if isinstance(node, LeafNode):
                return 1
            return (count_leaves_recursive(node.left) + 
                   count_leaves_recursive(node.right))
        
        if hasattr(self.model, 'tree_'):
            return count_leaves_recursive(self.model.tree_)
        return 0

class ModelInterpretability:
    """
    Tools for explaining decision tree predictions to business stakeholders.
    
    Essential for Amazon's customer obsession:
    - Clear explanations for automated decisions
    - Compliance with regulatory requirements
    - Building trust in ML systems
    """
    
    @staticmethod
    def explain_prediction(model, sample, feature_names):
        """
        Explain a single prediction by showing the decision path.
        
        Returns the exact sequence of decisions that led to the prediction.
        """
        if not hasattr(model, 'tree_'):
            raise ValueError("Model must have tree_ attribute")
        
        explanation = []
        node = model.tree_
        
        while isinstance(node, InternalNode):
            feature_name = feature_names[node.feature_idx]
            feature_value = sample[node.feature_idx]
            threshold = node.threshold
            
            if feature_value <= threshold:
                direction = "≤"
                explanation.append(f"{feature_name} {direction} {threshold:.3f}")
                node = node.left
            else:
                direction = ">"
                explanation.append(f"{feature_name} {direction} {threshold:.3f}")
                node = node.right
        
        # Final prediction
        prediction = node.prediction
        confidence = node.confidence
        
        return {
            'decision_path': explanation,
            'prediction': prediction,
            'confidence': confidence,
            'samples_in_leaf': node.samples
        }
    
    @staticmethod
    def business_rule_translation(rules_text):
        """
        Convert technical tree rules to business language.
        
        Example transformation:
        "feature_0 <= 3.5" → "Customer purchase amount <= $3,500"
        """
        # This would be customized based on feature meanings
        business_translations = {
            'feature_0': 'Customer Purchase Amount',
            'feature_1': 'Account Age (days)',
            'feature_2': 'Previous Orders Count',
            'feature_3': 'Risk Score'
        }
        
        translated_rules = rules_text
        for tech_name, business_name in business_translations.items():
            translated_rules = translated_rules.replace(tech_name, business_name)
        
        return translated_rules

class ScalabilityBenchmark:
    """
    Benchmark decision tree performance across different scales.
    
    Critical for capacity planning and SLA commitments.
    """
    
    @staticmethod
    def benchmark_training_time(max_samples=10000, step=1000):
        """
        Measure how training time scales with dataset size.
        """
        sample_sizes = range(step, max_samples + 1, step)
        training_times = []
        
        print("🔬 Training Time Scalability:")
        print(f"{'Samples':<10} {'Time (s)':<10} {'Time/Sample (ms)':<15}")
        print("-" * 40)
        
        for n_samples in sample_sizes:
            # Generate synthetic dataset
            X, y = make_classification(
                n_samples=n_samples,
                n_features=10,
                random_state=42
            )
            
            # Measure training time
            start_time = time.time()
            dt = DecisionTreeClassifier(random_state=42)
            dt.fit(X, y)
            training_time = time.time() - start_time
            
            training_times.append(training_time)
            time_per_sample = (training_time * 1000) / n_samples
            
            print(f"{n_samples:<10} {training_time:<10.3f} {time_per_sample:<15.3f}")
        
        return list(sample_sizes), training_times
    
    @staticmethod
    def benchmark_prediction_time(n_samples=1000, tree_depths=[3, 5, 10, 15]):
        """
        Measure prediction time vs tree complexity.
        """
        print(f"\n🚀 Prediction Time vs Tree Depth:")
        print(f"{'Depth':<8} {'Time (ms)':<12} {'Predictions/sec':<15}")
        print("-" * 40)
        
        # Generate test data
        X_test, _ = make_classification(
            n_samples=n_samples,
            n_features=10,
            random_state=42
        )
        
        for depth in tree_depths:
            # Train tree with specific depth
            X_train, y_train = make_classification(
                n_samples=5000,
                n_features=10,
                random_state=42
            )
            
            dt = DecisionTreeClassifier(max_depth=depth, random_state=42)
            dt.fit(X_train, y_train)
            
            # Measure prediction time
            start_time = time.time()
            predictions = dt.predict(X_test)
            prediction_time = time.time() - start_time
            
            time_ms = prediction_time * 1000
            predictions_per_sec = n_samples / prediction_time
            
            print(f"{depth:<8} {time_ms:<12.3f} {predictions_per_sec:<15.0f}")

# Demonstrate production features
print("\n🏭 PRODUCTION FEATURE DEMONSTRATION:")

# Create realistic dataset with some complexity
np.random.seed(42)
X_prod, y_prod = make_classification(
    n_samples=1000,
    n_features=8,
    n_informative=6,
    n_redundant=2,
    n_clusters_per_class=1,
    flip_y=0.05,
    random_state=42
)

# Add some missing values to simulate real-world data
missing_mask = np.random.random(X_prod.shape) < 0.05
X_prod[missing_mask] = np.nan

# Define business-meaningful feature names
feature_names = [
    'Purchase_Amount', 'Account_Age_Days', 'Previous_Orders',
    'Risk_Score', 'Geographic_Region', 'Device_Type',
    'Time_Since_Last_Order', 'Customer_Lifetime_Value'
]

# Train production model
prod_model = ProductionDecisionTree(
    criterion='gini',
    max_depth=6,
    min_samples_split=20,
    min_samples_leaf=10,
    random_state=42
)

prod_model.fit(X_prod, y_prod, feature_names=feature_names)

print(f"✅ Model trained successfully!")
print(f"Training time: {prod_model.training_stats['training_time']:.3f} seconds")
print(f"Tree depth: {prod_model.training_stats['tree_depth']}")

# Extract interpretable rules
print(f"\n📋 DECISION RULES (Top 3 levels):")
rules = prod_model.extract_rules(max_depth=3)
business_rules = ModelInterpretability.business_rule_translation(rules)
print(business_rules[:500] + "...")

# Feature importance report
print(f"\n🔍 FEATURE IMPORTANCE REPORT:")
importance_report = prod_model.get_feature_importance_report()
for item in importance_report[:5]:  # Top 5 features
    print(f"  {item['rank']}. {item['feature']}: {item['importance']:.3f}")

# Explain a single prediction
print(f"\n🔬 PREDICTION EXPLANATION:")
sample_idx = 0
sample = X_prod[sample_idx]
explanation = ModelInterpretability.explain_prediction(
    prod_model.model, sample, feature_names
)

print(f"Sample prediction: {explanation['prediction']}")
print(f"Confidence: {explanation['confidence']:.3f}")
print(f"Decision path:")
for i, step in enumerate(explanation['decision_path']):
    print(f"  {i+1}. {step}")

# Performance profiling
print(f"\n📊 PERFORMANCE PROFILE:")
profile_data = prod_model.performance_profile()
for key, value in profile_data.items():
    if key != 'training_stats':
        print(f"  {key}: {value}")

# Scalability benchmarks
print(f"\n⚡ SCALABILITY BENCHMARKS:")
benchmark = ScalabilityBenchmark()

# Training time benchmark (smaller scale for demo)
sample_sizes, times = benchmark.benchmark_training_time(max_samples=5000, step=1000)

# Prediction time benchmark
benchmark.benchmark_prediction_time(n_samples=1000)

print(f"\n💡 Production Insights:")
print(f"• Missing value handling: Automatic median imputation")
print(f"• Rule extraction: Ready for business stakeholder review")
print(f"• Performance monitoring: Built-in profiling and statistics")
print(f"• Scalability: Linear training time, constant prediction time")
print(f"• Interpretability: Full decision path explanation available")

print(f"\n✅ Production deployment ready!")

---

# 🎯 Interview Readiness Assessment

## **Critical Questions & Model Answers**

### **🧮 Mathematical Foundation Questions**

**Q1: "Derive the entropy formula and explain why we use log base 2."**

**Model Answer:**
```
Entropy H(S) = -Σ p_i * log₂(p_i)

Derivation from information theory:
- Information content I(x) = -log(p(x))
- Expected information = Σ p(x) * I(x) = -Σ p(x) * log(p(x))
- Log base 2 gives units in "bits" of information
- Alternative: natural log gives "nats", but doesn't affect split rankings
```

**Q2: "Compare Gini vs Entropy. When would you prefer each?"**

**Model Answer:**
- **Gini**: Faster (no log computation), range [0, 0.5], tends toward pure splits
- **Entropy**: More theoretically principled, range [0, log₂(c)], better probability estimates
- **Practice**: Usually give similar results, Gini preferred for speed
- **Choose Entropy**: When probability calibration matters (e.g., medical diagnosis)

---

### **⚙️ Algorithm Implementation Questions**

**Q3: "What's the time complexity of training a decision tree?"**

**Model Answer:**
```
Best case: O(n * m * log n)
- n = samples, m = features
- log n from tree depth (balanced tree)
- For each node: sort features O(n log n), try m features

Worst case: O(n² * m)
- Occurs with skewed splits (linear tree)
- Depth becomes O(n) instead of O(log n)

Space complexity: O(n) for tree storage
```

**Q4: "How do you handle categorical features with many categories?"**

**Model Answer:**
1. **Binary encoding**: Convert to multiple binary features
2. **Frequency encoding**: Replace with category frequency
3. **Target encoding**: Replace with mean target value (with regularization)
4. **Grouping**: Combine rare categories into "other"
5. **Hash encoding**: Use feature hashing for very high cardinality

---

### **🏭 Production & Business Questions**

**Q5: "Customer complains our fraud detection is biased. How do you investigate?"**

**Model Answer:**
1. **Audit feature importance**: Check if protected attributes have high importance
2. **Analyze decision paths**: Extract rules and review for discriminatory patterns
3. **Fairness metrics**: Calculate equalized odds, demographic parity
4. **Data investigation**: Check for proxy variables (zip code → race)
5. **Solution**: Fairness-aware tree algorithms, adversarial debiasing

**Q6: "How do you explain a tree's decision to non-technical stakeholders?"**

**Model Answer:**
```
Example explanation:
"The system flagged this transaction because:
1. Purchase amount ($5,000) exceeds the high-risk threshold ($3,000)
2. Account is only 15 days old (our model flags accounts < 30 days)
3. This combination has 85% fraud rate in historical data

Recommendation: Manual review required"
```

---

## 📚 **Comprehensive Summary**

### **Key Concepts Mastered:**

#### **1. Mathematical Foundations ✅**
- Information theory (entropy, information gain, Gini)
- Splitting criteria and bias issues
- Complexity analysis

#### **2. Algorithm Implementation ✅**
- Recursive tree construction
- Stopping criteria and pruning
- Feature selection and handling

#### **3. Advanced Techniques ✅**
- Ensemble integration (Random Forest, Boosting)
- Feature importance (multiple methods)
- Bias-variance analysis

#### **4. Production Deployment ✅**
- Scalability considerations
- Interpretability and explanation
- Real-world applications and pitfalls

---

## 🚀 **Amazon-Specific Applications**

### **Fraud Detection Pipeline:**
```python
# Multi-stage decision tree system
Stage 1: Fast screening (simple tree, <10ms)
├── Amount > $1000? → Stage 2
└── Amount ≤ $1000 → AUTO-APPROVE

Stage 2: Detailed analysis (complex tree, <100ms)
├── High-risk patterns → MANUAL_REVIEW
├── Medium-risk → ADDITIONAL_VERIFICATION
└── Low-risk → AUTO-APPROVE
```

### **Recommendation System Integration:**
```python
# User segmentation for recommendation strategy
Customer_Type_Tree:
├── Purchase_History > 100
│   ├── Electronics_Affinity → Tech_Recommendations
│   └── Fashion_Affinity → Style_Recommendations
└── Purchase_History ≤ 100
    ├── Price_Sensitive → Deal_Based_Recommendations
    └── Quality_Focused → Premium_Recommendations
```

---

## 📖 **Study Plan & Next Steps**

### **Week 1: Foundations**
- [ ] Implement entropy/Gini from scratch
- [ ] Code basic tree construction algorithm
- [ ] Practice complexity analysis

### **Week 2: Advanced Features**
- [ ] Implement pruning algorithms
- [ ] Code feature importance calculations
- [ ] Study ensemble methods integration

### **Week 3: Production Focus**
- [ ] Build production-ready tree class
- [ ] Practice explaining decisions
- [ ] Study real-world case studies

### **Interview Preparation:**
- [ ] Can implement tree from scratch in 45 minutes
- [ ] Can explain business impact of design choices
- [ ] Can debug common production issues
- [ ] Can communicate with non-technical stakeholders

---

## 🏆 **Final Assessment**

**You are ready for Amazon Applied Scientist interviews when you can:**

1. **✅ Code**: Implement decision tree from scratch with proper complexity analysis
2. **✅ Explain**: Translate technical concepts to business stakeholders
3. **✅ Scale**: Design production systems handling millions of decisions/day
4. **✅ Debug**: Identify and solve real-world ML pipeline issues
5. **✅ Innovate**: Propose improvements and new applications

### **🎯 Success Metrics:**
- **Technical Depth**: Master mathematical foundations
- **System Design**: Handle production scale and constraints
- **Business Acumen**: Connect ML decisions to customer value
- **Communication**: Explain complex concepts simply

---

**🚀 Ready to ace your Amazon Applied Scientist interview!**

*Remember: Amazon values customer obsession, ownership, and dive deep. Show how decision trees create customer value through interpretable, scalable ML systems.*

In [ ]:
# Final Comprehensive Visualization and Exercises

print("🎯 DECISION TREES LECTURE COMPLETE")
print("=" * 50)

# Create comprehensive visualization of decision tree concepts
fig = plt.figure(figsize=(16, 12))

# 1. Entropy vs Gini comparison
ax1 = plt.subplot(2, 3, 1)
p_values = np.linspace(0.01, 0.99, 100)
entropy_values = [-p * np.log2(p) - (1-p) * np.log2(1-p) for p in p_values]
gini_values = [2 * p * (1-p) for p in p_values]

ax1.plot(p_values, entropy_values, 'b-', linewidth=2, label='Entropy')
ax1.plot(p_values, gini_values, 'r-', linewidth=2, label='Gini')
ax1.set_xlabel('Probability of Class 1')
ax1.set_ylabel('Impurity')
ax1.set_title('Entropy vs Gini Impurity')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Information gain visualization
ax2 = plt.subplot(2, 3, 2)
# Simulate information gain for different thresholds
thresholds = np.linspace(0, 10, 50)
# Simulated information gain (would be calculated from real data)
info_gains = np.exp(-(thresholds - 5)**2 / 4) * 0.3

ax2.plot(thresholds, info_gains, 'g-', linewidth=2, marker='o', markersize=4)
best_threshold = thresholds[np.argmax(info_gains)]
ax2.axvline(best_threshold, color='red', linestyle='--', alpha=0.7, label=f'Best: {best_threshold:.1f}')
ax2.set_xlabel('Threshold Value')
ax2.set_ylabel('Information Gain')
ax2.set_title('Information Gain vs Threshold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Bias-Variance trade-off
ax3 = plt.subplot(2, 3, 3)
depths = range(1, 16)
bias = [1.0 / d for d in depths]  # Simplified: bias decreases with depth
variance = [d * 0.05 for d in depths]  # Simplified: variance increases with depth
total_error = [b + v + 0.1 for b, v in zip(bias, variance)]

ax3.plot(depths, bias, 'r-', linewidth=2, label='Bias²')
ax3.plot(depths, variance, 'b-', linewidth=2, label='Variance')
ax3.plot(depths, total_error, 'k-', linewidth=2, label='Total Error')
optimal_depth = depths[np.argmin(total_error)]
ax3.axvline(optimal_depth, color='green', linestyle='--', alpha=0.7, label=f'Optimal: {optimal_depth}')
ax3.set_xlabel('Tree Depth')
ax3.set_ylabel('Error')
ax3.set_title('Bias-Variance Trade-off')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Training time complexity
ax4 = plt.subplot(2, 3, 4)
sample_sizes = np.array([100, 500, 1000, 2000, 5000, 10000])
# Simulate O(n log n) complexity
training_times = sample_sizes * np.log(sample_sizes) / 1000

ax4.plot(sample_sizes, training_times, 'purple', linewidth=2, marker='s', markersize=6)
ax4.set_xlabel('Number of Samples')
ax4.set_ylabel('Training Time (arbitrary units)')
ax4.set_title('Training Time Complexity O(n log n)')
ax4.grid(True, alpha=0.3)

# 5. Feature importance comparison
ax5 = plt.subplot(2, 3, 5)
features = ['Amount', 'Age', 'History', 'Risk', 'Region', 'Device']
impurity_importance = [0.35, 0.25, 0.20, 0.15, 0.03, 0.02]
permutation_importance = [0.40, 0.22, 0.18, 0.12, 0.05, 0.03]

x_pos = np.arange(len(features))
width = 0.35

ax5.bar(x_pos - width/2, impurity_importance, width, label='Impurity-based', alpha=0.8)
ax5.bar(x_pos + width/2, permutation_importance, width, label='Permutation', alpha=0.8)
ax5.set_xlabel('Features')
ax5.set_ylabel('Importance')
ax5.set_title('Feature Importance Methods')
ax5.set_xticks(x_pos)
ax5.set_xticklabels(features, rotation=45)
ax5.legend()
ax5.grid(True, alpha=0.3)

# 6. Model performance vs complexity
ax6 = plt.subplot(2, 3, 6)
complexity = range(1, 21)
train_acc = [0.5 + 0.4 * (1 - np.exp(-c/3)) for c in complexity]
test_acc = [0.5 + 0.35 * (1 - np.exp(-c/3)) - 0.01 * c for c in complexity]

ax6.plot(complexity, train_acc, 'g-', linewidth=2, label='Training Accuracy', marker='o')
ax6.plot(complexity, test_acc, 'r-', linewidth=2, label='Testing Accuracy', marker='s')
best_complexity = complexity[np.argmax(test_acc)]
ax6.axvline(best_complexity, color='blue', linestyle='--', alpha=0.7, label=f'Optimal: {best_complexity}')
ax6.set_xlabel('Model Complexity')
ax6.set_ylabel('Accuracy')
ax6.set_title('Training vs Testing Performance')
ax6.legend()
ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Decision Trees: Complete Visual Guide', fontsize=16, y=0.98)
plt.show()

# Final interview coding exercises
print(f"\n🧮 INTERVIEW CODING EXERCISES:")
print(f"=" * 40)

print(f"\n1. Quick Implementation Challenge (15 minutes):")
print(f"   Implement binary classification tree with Gini criterion")

print(f"\n2. Complexity Analysis (10 minutes):")
print(f"   Derive time complexity for finding best split")

print(f"\n3. Production Scenario (15 minutes):")
print(f"   Design tree-based fraud detection for 1M transactions/day")

print(f"\n4. Business Communication (5 minutes):")
print(f"   Explain why a transaction was flagged to customer service")

# Success metrics summary
print(f"\n🏆 LECTURE COMPLETION SUMMARY:")
print(f"=" * 40)

concepts_covered = [
    "✅ Information Theory (Entropy, Gini, Information Gain)",
    "✅ Tree Construction Algorithm (ID3/CART)",
    "✅ Complexity Analysis (Time: O(n·m·log n), Space: O(n))",
    "✅ Pruning Techniques (Pre/Post, Cost Complexity)",
    "✅ Feature Importance (Impurity, Permutation, Drop-column)",
    "✅ Bias-Variance Trade-off Analysis",
    "✅ Ensemble Integration (Random Forest, Boosting)",
    "✅ Production Considerations (Scalability, Interpretability)",
    "✅ Real-world Applications (Fraud, Recommendations, A/B Testing)",
    "✅ Business Communication Strategies"
]

for concept in concepts_covered:
    print(f"  {concept}")

print(f"\n🎯 AMAZON APPLIED SCIENTIST READINESS:")
print(f"  ✅ Technical Mastery: Deep understanding of algorithms")
print(f"  ✅ System Design: Production-scale considerations")
print(f"  ✅ Business Impact: Customer value creation")
print(f"  ✅ Communication: Stakeholder explanation skills")

print(f"\n🚀 NEXT STEPS:")
print(f"  1. Practice implementing trees from scratch (3-5 times)")
print(f"  2. Review real Amazon case studies")
print(f"  3. Practice explaining decisions to business stakeholders")
print(f"  4. Study ensemble methods (Random Forest, XGBoost)")
print(f"  5. Learn tree-based feature selection techniques")

print(f"\n💡 INTERVIEW TIPS:")
print(f"  • Start with simple solution, then optimize")
print(f"  • Always consider edge cases (empty nodes, single class)")
print(f"  • Discuss trade-offs (bias-variance, speed-accuracy)")
print(f"  • Connect to business value and customer impact")
print(f"  • Be ready to implement core algorithm in 30-45 minutes")

print(f"\n🏁 DECISION TREES LECTURE SERIES COMPLETE!")
print(f"   You're now ready to tackle Amazon Applied Scientist interviews")
print(f"   with confidence in decision tree algorithms!")

# Quick self-assessment
def quick_assessment():
    """
    Quick self-assessment for interview readiness.
    """
    questions = [
        "Can you implement a decision tree from scratch in 45 minutes?",
        "Can you explain entropy vs Gini to a business stakeholder?",
        "Can you analyze time complexity of tree construction?",
        "Can you design a production fraud detection system?",
        "Can you explain why a specific prediction was made?",
        "Can you discuss bias-variance trade-offs in trees?",
        "Can you integrate trees with ensemble methods?",
        "Can you handle missing values and categorical features?"
    ]
    
    print(f"\n🎯 SELF-ASSESSMENT CHECKLIST:")
    print(f"Rate yourself 1-5 on each question:")
    for i, question in enumerate(questions, 1):
        print(f"  {i}. {question}")
    
    print(f"\n🏆 Target Score: 32+ out of 40 for interview readiness")

quick_assessment()

print(f"\n🎓 Congratulations on completing the Decision Trees lecture series!")
print(f"   You've gained comprehensive knowledge from mathematical foundations")
print(f"   to production deployment. You're ready for Amazon Applied Scientist interviews!")